### Corporate Finance: Group project #1
# BUILDING A RELATIVE VALUATION ALGORITHM
In this project, we will guide you on how to use Python to create a proper algorithm that outputs crucial information for the relative valuation process.

As you know, Relative Valuation (or Valuation via Multiples) is extremely useful for its simplicity, although this can also entail some problems. The underlying idea for this valuation is that similar firms should be trading (priced) at similar prices with respect to their multiples (e.g., PE, EV/EBITDA, EV/Sales, …)

Even though it is relatively easy to perform these calculations, provided that you have access to the data, we will create an algorithm to facilitate the task.

---
## 0. Import required packages
In this exercise, we will use [pandas](https://pandas.pydata.org/) and [plotly](https://plotly.com/).

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

---
## 1. Define the universe
We will consider a group of biopharma companies. This list contains their stock market tickers. All of them belong to the SP 500 index.

In [2]:
UNIVERSE = ['ABBV-US', 'AMGN-US', 'ABT-US', 'JNJ-US', 'MMM-US', 'MRK-US', 'PFE-US', 'TMO-US']

---
## 2. Read the SP 500 constituents
The excel file `listing.xlsx` constains the constituents of the SP500 including some company-level data we will need later.

In [3]:
listing = pd.read_excel('listing.xlsx', index_col='Id')
listing = listing.set_index('Symbol')
listing.tail()

,Date,Exchange,Security Type,Company Name,RBICS Economy,RBICS Sector,RBICS Subsector,Industry Group,Industry,Subindustry,Common Shares Outstanding - Security Level,Shares - Company Level,Market Value - Company Level
Symbol,,,,,,,,,,,,,
PARA-US,2022-09-16,NASDAQ,SHARE,Paramount Global Class B,Consumer Services,Media and Publishing Services,Media and Publishing Services,Entertainment and Programming Providers,Other Entertainment and Programming Providers,General Entertainment and Programming,608.42145,649.12690,15090.597279
CARR-US,2022-09-16,NYSE,SHARE,Carrier Global Corp.,Industrials,Industrial Manufacturing,Machinery Manufacturing,"Air, Liquid and Gas Control Equipment",Environmental Control Machinery/Equipment Prod...,"Heating, Ventilation and Air Conditioning Prod...",841.58344,841.58344,33999.972061
OTIS-US,2022-09-16,NYSE,SHARE,Otis Worldwide Corporation,Business Services,Business Services,Business Support Services,Facilities and Other Support Services,Facilities Support Services,Building Maintenance and Engineering Services,422.79443,422.79443,29143.220437
MTCH-US,2022-09-16,NASDAQ,SHARE,"Match Group, Inc.",Technology,Software and Consulting,Internet and Data Services,Consumer Data and Services,Information and News Content Providers and Sites,Other Classifieds and Directories Media and Sites,282.98645,282.98645,16373.596181
WBD-US,2022-09-16,NASDAQ,SHARE,"Warner Bros. Discovery, Inc. Series A",Consumer Services,Media and Publishing Services,Media and Publishing Services,Entertainment and Programming Providers,Television Cable and Broadcast Networks,US TV Cable and Broadcast Networks,2427.59280,2427.59280,31971.397011


---
## 3. Sunburst chart of industry and market capitalization
Let's generate a [sunburst chart](https://plotly.com/python/sunburst-charts/) of the SP 500 by industry and market capitalization. It is interactive!

In [4]:
fig = px.sunburst(listing,
                  path=['RBICS Economy', 'RBICS Sector'], 
                  values='Market Value - Company Level', 
                  width=800, height=800)
fig.show()

---
## 4. Read stock prices
The excel `prices.xlsx` contains daily price, return and volume data.

In [5]:
all_prices = pd.read_excel('prices.xlsx', index_col='Id')
all_prices.tail(3)

,symbol,date,Price,Adjusted Price,Price Change,Price Returns Dividends Excluded,Price Returns Dividends Received,Price Returns Dividends Reinvested,Volume
Id,,,,,,,,,
1998,TMO-US,2022-09-13,559.82,559.52,-3.344327,-3.344327,-3.344327,-3.344327,1642.870
1999,TMO-US,2022-09-14,558.13,558.13,-0.301886,-0.301886,-0.248295,-0.248295,1139.399
2000,TMO-US,2022-09-15,551.98,551.98,-1.101899,-1.101899,-1.101899,-1.101899,890.450


---
## 5. Extract prices for a single company
The table contains data in long-format that belongs to many companies at different dates. How can we extract the prices of a given company?  
Remember to set an index that is unique and representative of each row (the date).

In [6]:
# function
def get_prices(all_prices, ticker):
    df = all_prices.loc[all_prices['symbol'] == ticker]
    df = df.set_index('date')
    return df

# example: retrieve Johnson & Johnson price data
df = get_prices(all_prices, 'JNJ-US')
df.tail()

,symbol,Price,Adjusted Price,Price Change,Price Returns Dividends Excluded,Price Returns Dividends Received,Price Returns Dividends Reinvested,Volume
date,,,,,,,,
2022-09-09,JNJ-US,165.71,165.71,0.193489,0.193489,0.193489,0.193489,5501.863
2022-09-12,JNJ-US,165.64,165.64,-0.042248,-0.042248,-0.042248,-0.042248,6365.306
2022-09-13,JNJ-US,161.33,161.33,-2.602029,-2.602029,-2.602029,-2.602029,6726.360
2022-09-14,JNJ-US,164.66,164.66,2.064097,2.064097,2.064097,2.064097,9555.693
2022-09-15,JNJ-US,165.08,165.08,0.255072,0.255072,0.255072,0.255072,7174.495


---
## 6. Plot prices and volume of a company
Let's display in a single chart the prices and volume of a given company.  
The price chart is a [scatter plot](https://plotly.com/python/line-and-scatter/) and volume is a [bar plot](https://plotly.com/python/bar-charts/).

In [7]:
# function
def plot_prices(prices, title):
    # Create subplots, that is, one chart on top of another
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('Adjusted Price', 'Volume'), 
                        row_width=[0.2, 0.7])

    # top chart: prices
    fig.add_trace(go.Scatter(x = prices.index, y = prices['Adjusted Price'], name='Price'), 
                    row=1, col=1)

    # bottom chart: bar plot for volumes
    fig.add_trace(go.Bar(x=prices.index, y=prices['Volume'], marker_line_color='red', name='Volume'), row=2, col=1)

    # Do not show rangeslider plot 
    fig.update_layout(xaxis_rangeslider_visible=False, width=1200, height=600, title=title)
    fig.show()

# example: plot johnson & johnson prices and volume
# df is the DataFrame retrieved in the cell above
plot_prices(df, 'Prices of Johnson & Johnson')

---
## 7. Read Fundamental Data
Let's read the excel files containing the balance sheets, income statements and cash flows.  
(Cash flows are not used in the exercise).

In [8]:
all_balance_sheets = pd.read_excel('balance_sheets.xlsx', index_col='Id')
all_income_statements = pd.read_excel('income_statements.xlsx', index_col='Id')
all_cash_flows = pd.read_excel('cash_flows.xlsx', index_col='Id')

---
## 8. Multiples: DO YOUR MAGIC HERE!!
Insert below all the code you need to calculate the multiples!

$$PE=\frac{MarketCap}{Earnings_{TTM}}$$

$$\frac{EV}{EBITDA}=\frac{MarketCap + Debt - Cash}{EBITDA_{TTM}}$$

In [9]:
def get_ttm(df, symbol_key, value_key, date_key, ttm_key):
    """
    Calculates the TTM (trailing twelve months) for any given value and adds it as a new column

    If the date has less than 12 trailing months (<4 quarters) it fills in the missing values using the average
    of the available data

    df: DataFrame
    symbol_key: Name of the column that contains the company symbol
    value_key: Name of the column of the value we want the TTM of
    date_key: Name of the column that contains the dates used for the TTM
    ttm_key: Name of the output column (with the TTM value)
    """

    # Sort the DataFrame by the date key in ascending order
    df = df.sort_values(by=date_key)
    
    # Create a new DataFrame to store the TTM values
    ttm_df = pd.DataFrame()
    
    for index, row in reversed(list(df.iterrows())):
        # Extract the current date, value and symbol
        current_date = row[date_key]
        current_value = row[value_key]
        current_symbol = row[symbol_key]
        
        # Calculate the date 12 months ago
        ttm_date = current_date - pd.DateOffset(months=12)

        # Filter the dataframe to get only values from the current company
        search_space = df[df[symbol_key] == current_symbol]
        # Filter the dataframe to get only values in the TTM period
        ttm_data = search_space[(search_space[date_key] <= current_date) & (search_space[date_key] > ttm_date)]
        # Calculate the TTM value by summing the values within the TTM period
        ttm_value = ttm_data[value_key].sum()

        # If there are less than 4 records (less than 12 TTM), fill with average
        if len(ttm_data) < 4:
            ttm_average = ttm_data[value_key].mean()
            offset = 4 - len(ttm_data)
            ttm_value += offset*ttm_average

        # Add the TTM value to the new DataFrame
        ttm_row = pd.DataFrame({symbol_key: [current_symbol], date_key: [current_date], ttm_key: [ttm_value]})
        ttm_df = pd.concat([ttm_df, ttm_row], ignore_index=True)
    
    # Merge the TTM DataFrame with the original DataFrame based on (symbol, date)
    df = pd.merge(df, ttm_df, on=[symbol_key, date_key], how='left')
    
    return df

In [10]:
def merge_daily_quarterly(df_target, df_ref, symbol_target_key, date_target_key, symbol_ref_key, value_ref_key, date_ref_key):
    """
    Merges daily data with quarterly reports
    Each daily record is merged with the previous nearest quarterly record
    If the daily date is 27/06/2001, and I have quarterly records for 30/06/2001, 31/03/2001 and 31/12/2000 it will merge with the 
    value of 31/03/2001 (the preivous nearest)

    It returns the daily dataframe with the merged quarterly data
        
    df_target: Data frame with daily values
    df_ref: Data frame with quarterly values
    symbol_target_key: Column with the symbol of the target DF
    date_target_key: Column with the daily date of the target DF
    symbol_ref_key: Column with the symbol of the ref DF
    value_ref_key: Column with the value to merge
    date_ref_key: Column with the quarterly date
    """
    
    # Create a new DataFrame to store the last available values
    nearest_previous_df = pd.DataFrame()

    # Sort quarterly DF by date (ascending)
    df_ref = df_ref.sort_values(by=date_ref_key)
    
    for index, row in df_target.iterrows():
        # Get current date and symbol
        current_date = row[date_target_key]
        current_symbol = row[symbol_target_key]

        # Get only the values from the current symbol
        search_space = df_ref[df_ref[symbol_ref_key] == current_symbol]
        # Search the nearest previous date
        index = search_space[date_ref_key].searchsorted(current_date)

        # Select the row with the nearest previous date
        nearest_previous_data = search_space.iloc[index-1]

        # Add the nearest previous value to the new DataFrame
        nearest_previous_row = pd.DataFrame({
            symbol_target_key: [current_symbol], 
            date_target_key: [current_date], 
            value_ref_key: [nearest_previous_data[value_ref_key]]
        })
        nearest_previous_df = pd.concat([nearest_previous_df, nearest_previous_row], ignore_index=True)

    # Merge the nearest_previous_df DataFrame with the original DataFrame based on (symbol, daily date)
    df = pd.merge(df_target, nearest_previous_df, on=[symbol_target_key, date_target_key], how='left')
    
    return df

In [11]:
def pivot_and_aggregate(df, ratio):
    """
    Create pivot table for plots and add aggreate figures
    It returns the pivoted table.

    df: Data frame to pivot
    ratio: Column with the ratio to plot
    """
    
    df_pivot = df.pivot(columns="symbol", index="date", values=ratio)
    columns = df_pivot.columns
    df_pivot["low"] = df_pivot[columns].min(axis=1)
    df_pivot["high"] = df_pivot[columns].max(axis=1)
    df_pivot["median"] = df_pivot[columns].median(axis=1)
    return df_pivot

In [12]:
# Create a DF with Earnings Per Share TTM
earnings_ttm = get_ttm(all_income_statements, "symbol", "Earnings Per Share", "Fiscal Date", "EPS TTM")
earnings_ttm.head()

,symbol,Sales,COGS incl. Depreciation & Amortization,Gross Income,"Selling, General & Administrative Expense",EBIT,Nonoperating Income (Net) - Total,Interest Expense on Debt,Unusual Expense,Pretax Income,...,Preferred Dividends,EBITDA,Depreciation & Amortization,Enterprise Value,Enterprise Value to EBITDA,Earnings Per Share,Sales per Share,Filing Date,Fiscal Date,EPS TTM
0,AMGN-US,4521.0,991.0,3530.0,2050.0,1464.0,97.0,259.0,98.0,1204.0,...,0.0,1982.0,518.0,106947.12,14.311136,1.4000,5.886719,2014-04-30,2014-03-31,5.6000
1,ABBV-US,4563.0,1088.0,3475.0,2030.0,1445.0,-5.0,70.0,84.0,1286.0,...,0.0,1644.0,199.0,88008.60,12.046072,0.6100,2.835923,2014-05-09,2014-03-31,2.4400
2,MRK-US,10265.0,3784.0,6481.0,4215.0,2266.0,265.0,188.0,252.0,2091.0,...,0.0,4020.0,1754.0,178887.67,10.741424,0.5700,3.455066,2014-05-08,2014-03-31,2.2800
3,PFE-US,11353.0,3082.0,8271.0,4637.0,3634.0,525.0,331.0,991.0,2847.0,...,0.0,5090.0,1456.0,209872.88,9.353041,0.3491,1.753088,2014-05-08,2014-03-31,1.3964
4,MMM-US,7831.0,4036.0,3795.0,2084.0,1711.0,15.0,70.0,-32.0,1688.0,...,0.0,2061.0,350.0,95741.67,11.801019,1.7900,11.610082,2014-05-01,2014-03-31,7.1600


In [13]:
# Create a DF with EBITDA TTM
ebitda_ttm = get_ttm(all_income_statements, "symbol", "EBITDA", "Fiscal Date", "EBITDA TTM")
ebitda_ttm.head()

,symbol,Sales,COGS incl. Depreciation & Amortization,Gross Income,"Selling, General & Administrative Expense",EBIT,Nonoperating Income (Net) - Total,Interest Expense on Debt,Unusual Expense,Pretax Income,...,Preferred Dividends,EBITDA,Depreciation & Amortization,Enterprise Value,Enterprise Value to EBITDA,Earnings Per Share,Sales per Share,Filing Date,Fiscal Date,EBITDA TTM
0,AMGN-US,4521.0,991.0,3530.0,2050.0,1464.0,97.0,259.0,98.0,1204.0,...,0.0,1982.0,518.0,106947.12,14.311136,1.4000,5.886719,2014-04-30,2014-03-31,7928.0
1,ABBV-US,4563.0,1088.0,3475.0,2030.0,1445.0,-5.0,70.0,84.0,1286.0,...,0.0,1644.0,199.0,88008.60,12.046072,0.6100,2.835923,2014-05-09,2014-03-31,6576.0
2,MRK-US,10265.0,3784.0,6481.0,4215.0,2266.0,265.0,188.0,252.0,2091.0,...,0.0,4020.0,1754.0,178887.67,10.741424,0.5700,3.455066,2014-05-08,2014-03-31,16080.0
3,PFE-US,11353.0,3082.0,8271.0,4637.0,3634.0,525.0,331.0,991.0,2847.0,...,0.0,5090.0,1456.0,209872.88,9.353041,0.3491,1.753088,2014-05-08,2014-03-31,20360.0
4,MMM-US,7831.0,4036.0,3795.0,2084.0,1711.0,15.0,70.0,-32.0,1688.0,...,0.0,2061.0,350.0,95741.67,11.801019,1.7900,11.610082,2014-05-01,2014-03-31,8244.0


In [14]:
# To calculate PE we need (Stock Price * N of Shares) / (EPS TTM / N of shares) = Stock Price / EPS TTM
# We need a dataframe that has for, each Stock Price the corresponding EPS TTM for that quarter
# We can calculate it using merge_daily_quarterly
prices_eps_df = merge_daily_quarterly(
    all_prices,
    earnings_ttm,
    "symbol",
    "date",
    "symbol",
    "EPS TTM",
    "Fiscal Date"
)
prices_eps_df.head()

,symbol,date,Price,Adjusted Price,Price Change,Price Returns Dividends Excluded,Price Returns Dividends Received,Price Returns Dividends Reinvested,Volume,EPS TTM
0,ABBV-US,2014-10-06,57.81,41.329860,-1.499397,-1.499397,-1.499397,-1.499397,7359.936,2.133333
1,ABBV-US,2014-10-07,56.65,40.500546,-2.006572,-2.006572,-2.006572,-2.006572,8924.598,2.133333
2,ABBV-US,2014-10-08,58.34,41.708775,2.983224,2.983224,2.983224,2.983224,10917.240,2.133333
3,ABBV-US,2014-10-09,56.72,40.550594,-2.776825,-2.776825,-2.776825,-2.776825,11913.150,2.133333
4,ABBV-US,2014-10-10,54.97,39.592644,-3.085333,-3.085333,-2.344853,-2.344853,12108.840,2.133333


In [15]:
# To calculate Market Cap we need the number of shares sold for each company, for each stock price
# Create a new Data Frame that has, for each (symbol, date) the number of shares of the company
# NOTE: Using Outstanding Shares (change for Shares - Company Level if necessary)
market_cap_df = pd.merge(all_prices, listing[["Common Shares Outstanding - Security Level"]], left_on=["symbol"], right_on=["Symbol"], how='left')

# Now we can calculate Market Cap directly for each (symbol, date)
market_cap_df["Market Cap"] = market_cap_df["Adjusted Price"] * market_cap_df["Common Shares Outstanding - Security Level"]
market_cap_df.head()

,symbol,date,Price,Adjusted Price,Price Change,Price Returns Dividends Excluded,Price Returns Dividends Received,Price Returns Dividends Reinvested,Volume,Common Shares Outstanding - Security Level,Market Cap
0,ABBV-US,2014-10-06,57.81,41.329860,-1.499397,-1.499397,-1.499397,-1.499397,7359.936,1768.0966,73075.184944
1,ABBV-US,2014-10-07,56.65,40.500546,-2.006572,-2.006572,-2.006572,-2.006572,8924.598,1768.0966,71608.877681
2,ABBV-US,2014-10-08,58.34,41.708775,2.983224,2.983224,2.983224,2.983224,10917.240,1768.0966,73745.143268
3,ABBV-US,2014-10-09,56.72,40.550594,-2.776825,-2.776825,-2.776825,-2.776825,11913.150,1768.0966,71697.367379
4,ABBV-US,2014-10-10,54.97,39.592644,-3.085333,-3.085333,-2.344853,-2.344853,12108.840,1768.0966,70003.619241


In [16]:
# To calculate EV/EBITDA we need for each (symbol,daily date), EBITDA TTM, Cash and Debt which are quarterly values
# Use merge_daily_quarterly to solve it
market_cap_df = merge_daily_quarterly(
    market_cap_df,
    ebitda_ttm,
    "symbol",
    "date",
    "symbol",
    "EBITDA TTM",
    "Fiscal Date"
)
print(len(market_cap_df))

16008


In [17]:
# NOTE: Using Net Change in Cash as Cash
market_cap_df = merge_daily_quarterly(
    market_cap_df,
    all_cash_flows,
    "symbol",
    "date",
    "symbol",
    "Net Change in Cash",
    "Fiscal Date"
)
print(len(market_cap_df))

16008


In [18]:
# NOTE: Using ST/Debt and LT/Debt as Debt
all_balance_sheets["Debt"] = all_balance_sheets["ST Debt & Curr. Portion LT Debt"] + all_balance_sheets["Long-Term Debt"]
market_cap_df = merge_daily_quarterly(
    market_cap_df,
    all_balance_sheets,
    "symbol",
    "date",
    "symbol",
    "Debt",
    "Fiscal Date"
)
print(len(market_cap_df))

16008


In [19]:
# Create a subset of the original DF with only the necessary data and calculate PE ratio
pe_df = prices_eps_df.loc[:,["symbol", "date", "Adjusted Price", "EPS TTM"]]
pe_df["PE"] = pe_df["Adjusted Price"] / pe_df["EPS TTM"]
pe_df.head()

,symbol,date,Adjusted Price,EPS TTM,PE
0,ABBV-US,2014-10-06,41.329860,2.133333,19.373372
1,ABBV-US,2014-10-07,40.500546,2.133333,18.984631
2,ABBV-US,2014-10-08,41.708775,2.133333,19.550988
3,ABBV-US,2014-10-09,40.550594,2.133333,19.008091
4,ABBV-US,2014-10-10,39.592644,2.133333,18.559052


In [20]:
# Create a subset of the original DF with only the necessary data and calculate EV/EBITDA ratio
ev_ebitda_df = market_cap_df.loc[:,["symbol", "date","Market Cap", "EBITDA TTM", "Net Change in Cash", "Debt"]]
ev_ebitda_df["ev/ebitda"] = (ev_ebitda_df["Market Cap"] + ev_ebitda_df["Debt"] - ev_ebitda_df["Net Change in Cash"]) / ev_ebitda_df["EBITDA TTM"]
ev_ebitda_df.head()

,symbol,date,Market Cap,EBITDA TTM,Net Change in Cash,Debt,ev/ebitda
0,ABBV-US,2014-10-06,73075.184944,6869.333333,-1367.0,14792.0,12.990225
1,ABBV-US,2014-10-07,71608.877681,6869.333333,-1367.0,14792.0,12.776768
2,ABBV-US,2014-10-08,73745.143268,6869.333333,-1367.0,14792.0,13.087754
3,ABBV-US,2014-10-09,71697.367379,6869.333333,-1367.0,14792.0,12.789650
4,ABBV-US,2014-10-10,70003.619241,6869.333333,-1367.0,14792.0,12.543083


In [21]:
# Create dataframe to plot data
pe_plot = pivot_and_aggregate(pe_df, "PE")
pe_plot.head()

symbol,ABBV-US,ABT-US,AMGN-US,JNJ-US,MMM-US,MRK-US,PFE-US,TMO-US,low,high,median
date,,,,,,,,,,,
2014-10-06,19.373372,30.337513,16.528643,13.187275,14.605952,21.242446,12.733920,27.401760,12.733920,30.337513,17.951008
2014-10-07,18.984631,29.692935,16.338864,12.876646,14.329107,20.843134,12.568034,26.854318,12.568034,29.692935,17.661748
2014-10-08,19.550988,30.179928,16.824121,13.193563,14.717313,21.456374,12.825595,27.628918,12.825595,30.179928,18.187555
2014-10-09,19.008091,29.814682,16.525043,12.837659,14.429022,21.110534,12.655343,26.763464,12.655343,29.814682,17.766567
2014-10-10,18.559052,29.908281,16.502222,12.730764,13.928420,20.914433,12.716457,26.456802,12.716457,29.908281,17.530637


In [22]:
# Create dataframe to plot data
ev_ebitda_plot = pivot_and_aggregate(ev_ebitda_df, "ev/ebitda")
ev_ebitda_plot.head()

symbol,ABBV-US,ABT-US,AMGN-US,JNJ-US,MMM-US,MRK-US,PFE-US,TMO-US,low,high,median
date,,,,,,,,,,,
2014-10-06,12.990225,16.275790,10.325744,8.971594,8.098939,7.795416,7.293050,15.557334,7.293050,16.275790,9.648669
2014-10-07,12.776768,15.969641,10.250166,8.774190,7.961837,7.676685,7.221500,15.321011,7.221500,15.969641,9.512178
2014-10-08,13.087754,16.200944,10.443416,8.975590,8.154089,7.859025,7.332591,15.655395,7.332591,16.200944,9.709503
2014-10-09,12.789650,16.027466,10.324310,8.749415,8.011318,7.756193,7.259158,15.281790,7.259158,16.027466,9.536862
2014-10-10,12.543083,16.071922,10.315221,8.681483,7.763404,7.697885,7.285518,15.149408,7.285518,16.071922,9.498352


---
## 9. The visualization code, as promised
Given a `DataFrame` with the correct format and structure, this function will display:
- High band
- Median
- Low band
- The multiple(s) of the company(ies) selected in the legend on the right.

In [23]:
def plot_multiple(name, universe, data):

    fig = go.Figure()
    
    fig.add_trace(
        go.Scatter(x = data.index,
                   y = data['median'],
                   line = {'dash': 'solid', 'width':5},
                   line_color = 'gray',
                   name = 'median',
                   opacity = 0.25)
    )
    
    for ticker in universe:
        fig.add_trace(
            go.Scatter(x = data.index,
                       y = data[ticker],
                       #line_color = 'red',
                       visible='legendonly',
                       name = ticker)
        )

    # Upper Bound
    fig.add_trace(
        go.Scatter(x = data.index,
                   y = data['high'],
                   line_color = 'lightgray',
                   line = {'dash': 'dot', 'width':1},                   
                   name = 'upper band',
                   opacity = 0.25)
    )

    # Lower Bound fill in between with parameter 'fill': 'tonexty'
    fig.add_trace(
        go.Scatter(x = data.index,
                   y = data['low'],
                   line_color = 'lightgray',
                   line = {'dash': 'dot', 'width':1},
                   fill = 'tonexty',
                   name = 'lower band',
                   opacity = 0.25)
    )

    # use percentages in axes
    fig.update_layout(
        width=1200,
        height=600,
        title=name,
        xaxis_title='time',
        yaxis_title=name,
    )

    fig.show()

Plot multiple: $\frac{EV}{EBITDA}$

In [24]:
# example
plot_multiple('ev/ebitda', UNIVERSE, ev_ebitda_plot)

Plot multiple: $PE=\frac{Price}{Earnings}$

In [25]:
# example
plot_multiple('PE', UNIVERSE, pe_plot)